### Welcome to Week 6 Day 3!

Let's experiment with a bunch more MCP Servers

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [2]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/justin.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

In [ ]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Justin. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
# model = "gpt-4.1-mini"
from dotenv import load_dotenv
from agents import Agent, Runner, trace
import os
from openai import OpenAI, AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled
client = AsyncOpenAI(
            api_key=os.getenv("GEMINI_API_KEY"),
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
        )

model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash-lite",
    openai_client=client
)

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

It's great to meet you, Justin! It sounds like you're quite involved in the world of AI agents, both as an LLM engineer and as an educator. The MCP protocol you mentioned for connecting agents with tools, resources, and prompt templates sounds like a powerful way to enhance AI agent capabilities.

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [5]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Justin. What do you know about me?")
    display(Markdown(result.final_output))

You are an LLM engineer and you teach a course about AI Agents.

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service

### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [9]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

ValidationError: 1 validation error for StdioServerParameters
env.BRAVE_API_KEY
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [ ]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### As usual, check out the trace:

https://platform.openai.com/traces

## And now the third type: running remotely

It's actually really hard to find a "remote MCP server" aka "hosted MCP server" aka "managed MCP server".

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.

Anthropic lists some remote MCP servers, but these are for paid applications with business users:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlare has tooling for you to create and deploy your own remote MCP servers, but this does not seem to be a common practice:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/


# And back to the 2nd type: the Polygon.io MCP Server

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">PLEASE READ!!-</h2>
            <span style="color:#ff7800;">This service for financial market data has both a FREE plan and a PAID plan, and we can use either depending on your appetite.
            </span>
        </td>
    </tr>
</table>

## NEW SECTION: Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service (the paid will be totally optional, of course!)

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

In [ ]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [ ]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

### Wrapped into a python module that caches end of day prices

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.


In [ ]:
from market import get_share_price
get_share_price("AAPL")

In [ ]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [ ]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [ ]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Polygon.io Part 2: Paid Plan - Totally Optional!

If you are interested, you can subscribe to the monthly plan to get more up to date market data, and unlimited API calls.

If you do wish to do this, then it also makes sense to use the full MCP server that Polygon.io has released, to take advantage of all their functionality.



In [ ]:

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools


### Wow that's a lot of tools!

Let's try them out - hopefully the sheer number of tools doesn't overwhelm gpt-4o-mini!

With the $29 monthly plan, we don't have access to some of the APIs, so I've needed to specify which APIs can be called.

If you've splashed out on a bigger plan, feel free to remove my extra constraint..

In [ ]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Setting up your .env file

If you do decide to have a paid plan, please add this to your .env file to indicate:

`POLYGON_PLAN=paid`

And if you decide to go all the way for the realtime API, then please do:

`POLYGON_PLAN=realtime`

In [ ]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("You've chosen to subscribe to the paid Polygon plan, so the code will look at prices on a 15 min delay")
elif is_realtime_polygon:
    print("Wowzer - you've chosen to subscribe to the realtime Polygon plan, so the code will look at realtime prices")
else:
    print("According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices")

## And that's it for today!

I've removed the part of this lab that uses the "Financial Datasets" mcp server, because it's inferior - more expensive with fewer APIs.

And this way we get to use the same provider for Free and Paid APIs.

But if you want to see the code, just look in the git history for a prior version.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Explore MCP server marketplaces and integrate your own, using all 3 approaches.
            </span>
        </td>
    </tr>
</table>